In [1]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.metrics import f1_score, accuracy_score, roc_curve, auc, recall_score, precision_score

from torch_geometric.nn import GCNConv
from torch.nn import Linear

2.0.1+cpu


In [2]:
# Lädt Daten
usu_graph = torch.load('Graph/usu_graph.pth')
usu_graph_oversampled_easy = torch.load('Graph/syntetic_data.pth')

In [17]:
# GCN
class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        torch.manual_seed(12345)
        self.conv1 = GCNConv(25, 4)
        self.bn1 = torch.nn.BatchNorm1d(4)
        self.dropout1 = torch.nn.Dropout(0.5)
        self.conv2 = GCNConv(4, 4)
        self.bn2 = torch.nn.BatchNorm1d(4)
        self.dropout2 = torch.nn.Dropout(0.5)
        self.conv3 = GCNConv(4, 2)
        self.bn3 = torch.nn.BatchNorm1d(2)
        self.dropout3 = torch.nn.Dropout(0.5)
        self.classifier = Linear(2, 2)

    def forward(self, x, edge_index):
        h = self.conv1(x, edge_index)
        h = self.bn1(h)
        h = h.tanh()
        h = self.dropout1(h)
        h = self.conv2(h, edge_index)
        h = self.bn2(h)
        h = h.tanh()
        h = self.dropout2(h)
        h = self.conv3(h, edge_index)
        h = self.bn3(h)
        h = h.tanh()
        h = self.dropout3(h)

        out = self.classifier(h)

        return out, h

model = GCN()
print(model)

GCN(
  (conv1): GCNConv(25, 4)
  (bn1): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout1): Dropout(p=0.5, inplace=False)
  (conv2): GCNConv(4, 4)
  (bn2): BatchNorm1d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout2): Dropout(p=0.5, inplace=False)
  (conv3): GCNConv(4, 2)
  (bn3): BatchNorm1d(2, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout3): Dropout(p=0.5, inplace=False)
  (classifier): Linear(in_features=2, out_features=2, bias=True)
)


In [16]:
model = GCN()

weight_class_0 = 1.0
weight_class_1 = 5.0
class_weights = torch.Tensor([weight_class_0, weight_class_1])

criterion = torch.nn.CrossEntropyLoss(weight=class_weights)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

model.train()

def calculate_importance(h):
    # Implementieren Sie Ihre Metrik zur Berechnung der Wichtigkeit hier.
    # Hier verwenden wir einfach die L2-Norm der Hidden-Zustände als Beispiel.
    importance = torch.norm(h, dim=1)
    return importance

def check_pareto(importance):
    # Sortieren Sie die Wichtigkeit absteigend.
    sorted_indices = torch.argsort(importance, descending=True)
    cumulative_sum = torch.cumsum(importance[sorted_indices], dim=0)
    threshold = 0.8 * cumulative_sum[-1]

    # Finden Sie die Anzahl der Elemente, die das Pareto-Prinzip erfüllen.
    pareto_elements = (cumulative_sum >= threshold).nonzero()

    # Überprüfen Sie, ob das Pareto-Prinzip zutrifft und geben Sie die Ergebnisse aus.
    if len(pareto_elements) <= 0.2 * len(importance):
        print("Das Pareto-Prinzip trifft zu.")
    else:
        print("Das Pareto-Prinzip trifft nicht zu.")

def train(data):
    optimizer.zero_grad()
    out, h = model(data.x, data.edge_index)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])

    # Berechnen Sie die Wichtigkeit der Elemente basierend auf den Hidden-Zuständen.
    importance = calculate_importance(h)

    # Überprüfen Sie das Pareto-Prinzip.
    check_pareto(importance)

    loss.backward()
    optimizer.step()

    return loss, h

for epoch in range(150):
    loss, h = train(usu_graph)
    print(f'Epoch: {epoch}, Loss: {loss}')

Das Pareto-Prinzip trifft nicht zu.
Epoch: 0, Loss: 0.9451559782028198
Das Pareto-Prinzip trifft nicht zu.
Epoch: 1, Loss: 0.9339910745620728
Das Pareto-Prinzip trifft nicht zu.
Epoch: 2, Loss: 0.9260878562927246
Das Pareto-Prinzip trifft nicht zu.
Epoch: 3, Loss: 0.9077959656715393
Das Pareto-Prinzip trifft nicht zu.
Epoch: 4, Loss: 0.8949931859970093
Das Pareto-Prinzip trifft nicht zu.
Epoch: 5, Loss: 0.8849787712097168
Das Pareto-Prinzip trifft nicht zu.
Epoch: 6, Loss: 0.8755815029144287
Das Pareto-Prinzip trifft nicht zu.
Epoch: 7, Loss: 0.8649537563323975
Das Pareto-Prinzip trifft nicht zu.
Epoch: 8, Loss: 0.8592509627342224
Das Pareto-Prinzip trifft nicht zu.
Epoch: 9, Loss: 0.8498400449752808
Das Pareto-Prinzip trifft nicht zu.
Epoch: 10, Loss: 0.8405616879463196
Das Pareto-Prinzip trifft nicht zu.
Epoch: 11, Loss: 0.8331317901611328
Das Pareto-Prinzip trifft nicht zu.
Epoch: 12, Loss: 0.8191952109336853
Das Pareto-Prinzip trifft nicht zu.
Epoch: 13, Loss: 0.8127669095993042
Da